# 💬 Chat-Commerce 🛒

## Setup

In [ ]:
# Install packages
%pip install python-dotenv langchain llama-index

In [ ]:
from dotenv import load_dotenv

# Load environment values
load_dotenv()

In [ ]:
import logging
import sys

# Turn off unwanted logs
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

## Index E-Commerce Data

In [ ]:
from langchain import OpenAI

# OpenAI LLM for indexing
llm_index = OpenAI(temperature=0)

In [ ]:
from pathlib import Path
from llama_index import download_loader

# Read the data from CSV
PagedCSVReader = download_loader("PagedCSVReader")
loader = PagedCSVReader()
documents = loader.load_data(file=Path('./data/home-and-garden.csv'))

In [ ]:
from llama_index import GPTSimpleVectorIndex, LLMPredictor

# Index the data and store the embeddings
# NOTE: Run this only if you want to re-index
llm_predictor = LLMPredictor(llm=llm_index)
index_data = GPTSimpleVectorIndex(documents, llm_predictor=llm_predictor)
index_data.save_to_disk("./index/index.json")

In [ ]:
from llama_index import GPTSimpleVectorIndex

# Load the embeddings from existing index
index_data = GPTSimpleVectorIndex.load_from_disk('./index/index.json')

## Run Queries

In [ ]:
from langchain.llms import OpenAIChat

# Initialize OpenAI LLM for chat
llm_chat = OpenAIChat(temperature=0)

In [ ]:
from langchain.agents import Tool

# Create a langchain 'Tool'
tools = [
    Tool(
        name = "GPT Index",
        func=lambda q: str(index_data.query(q)),
        description="useful for when you want to answer questions about products available in the store. The store sells furnitures and items for home and garden",
        return_direct=True
    ),
]

In [ ]:
from llama_index import GPTListIndex
from langchain.agents import initialize_agent
from llama_index.langchain_helpers.memory_wrapper import GPTIndexChatMemory

# Index to be used for chat history
index_chat_history = GPTListIndex([])

# chat history
memory = GPTIndexChatMemory(
    index=index_chat_history, 
    memory_key="chat_history", 
    query_kwargs={"response_mode": "compact"},
    # return_source returns source nodes instead of querying index
    return_source=True,
    # return_messages returns context in message format
    return_messages=True
)

# create an agent
agent_chain = initialize_agent(tools, llm=llm_chat, agent="conversational-react-description", memory=memory)

In [ ]:
agent_chain.run(input="hi, i am bob")

In [ ]:
agent_chain.run(input="what are the products available?")

In [ ]:
agent_chain.run(input="I'm looking for some furniture to use in my garden")

In [ ]:
agent_chain.run(input="Great. Can you pick me one table?")

In [ ]:
agent_chain.run(input="Yes also show me a picture of it")